In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# Ping the website and grab data
link = 'https://toppubs.smedian.com/'
page_response = requests.get(link, timeout=1000)
page_content = BeautifulSoup(page_response.content)

# Loop to go through each row of top publications table and store data in lists
pub_names = []
pub_subs = []
pub_growth = []
pub_detail = []
for row in page_content.find_all('div', class_='row')[1:2]:
    pub = row.find_all('a', class_='link link--primary u-accentColor--hoverTextNormal ng-binding')
    detail = row.find_all('a', class_='link u-colorLightGrey u-fontWeight400 u-fontSize12px ng-binding')
    if pub != []:
        pub_names.append(pub[0].text.split('\n')[1])
        pub_subs.append(detail[0].text.split('\n')[1])
        if len(detail) > 2:
            pub_growth.append(detail[2].text.split('\n')[1].split(' ')[-1])
        else:
            pub_growth.append(0)
        pub_detail.append(row.find_all('p')[0].text)

# Store data in a dataframe
pub_df = pd.DataFrame()
pub_df['name'] = pub_names
pub_df['subscribers'] = pub_subs
pub_df['growth'] = pub_growth
pub_df['detail'] = pub_detail

# Do the same thing for fastest growing Medium publications page
link = 'https://toppubs.smedian.com/growth'
page_response = requests.get(link, timeout=1000)
page_content = BeautifulSoup(page_response.content)

pub_names = []
pub_subs = []
pub_growth = []
pub_detail = []
for row in page_content.find_all('div', class_='row')[1:]:
    pub = row.find_all('a', class_='link link--primary u-accentColor--hoverTextNormal ng-binding')
    detail = row.find_all('a', class_='link u-colorLightGrey u-fontWeight400 u-fontSize12px ng-binding')
    if pub != []:
        pub_names.append(pub[0].text.split('\n')[1])
        pub_subs.append(detail[0].text.split('\n')[1])
        if len(detail) > 2:
            pub_growth.append(detail[2].text.lstrip().split(' ')[2])
        else:
            pub_growth.append(0)
        pub_detail.append(row.find_all('p')[0].text)
        
growth_df = pd.DataFrame()
growth_df['name'] = pub_names
growth_df['subscribers'] = pub_subs
growth_df['growth'] = pub_growth
growth_df['detail'] = pub_detail

new_pubs = []
for i in growth_df['name']:
    if sum(i == pub_df['name']) == 0:
        new_pubs.append(i)

In [3]:
master_df = pd.concat([pub_df, growth_df[growth_df.name.isin(new_pubs)]])
master_df['subscribers'] = [int(i.replace(',','')) for i in master_df['subscribers']]
master_df['growth'] = [float(i) for i in master_df['growth']]

In [4]:
master_df

,name,subscribers,growth,detail
0,freeCodeCamp.org,603567,287.8,This is no longer updated. Go to https://freec...
0,Towards Data Science,275225,443.3,"Sharing concepts, ideas, and codes."
1,The Startup,506214,378.6,"Medium's largest active publication, followed ..."
3,UX Collective,291588,218.3,We believe designers are thinkers as much as t...
4,Better Humans,332235,210.3,Better Humans is a collection of the world's m...
5,The Coinbase Blog,340690,186.1,Learn about working at Coinbase: https://www.c...
6,UX Planet,219853,163.1,UX Planet is a one-stop resource for everythin...
7,Personal Growth,440811,141.5,Sharing our ideas and experiences.
8,Human Parts,127132,138.2,What it means to be
9,Mission.org,535070,132.9,A network of business & tech podcasts designed...
